# Proceso  de EDA para estudiar los datos, y preparacion para hacer un modelo de recomendacion

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_splitSV
from sklearn.preprocessing import LabelEncoder
import pickle
from surprise import SVD
from surprise.model_selection import GridSearchCV
from surprise import accuracy



## Steam Games

In [ ]:
steam_games = pd.read_csv('./datasets/steam_games.csv')

In [ ]:
steam_games['Year']

In [ ]:
steam_games.info()

In [ ]:
fig,ax = plt.subplots(1,3)
sns.boxplot(data=steam_games,y='price',ax = ax[0])
sns.boxplot(data=steam_games,y='price',ax = ax[1])
sns.boxplot(data=steam_games,y='price',ax = ax[2])
ax[0].set_ylim(0,100)
ax[1].set_ylim(0,400)
ax[2].set_ylim(0,800)
plt.show()

### En el grafico se puede ver que la columna price tiene muchos valores atipicos que superan los 20 y llegan a los 1000, sin embargo en una app de juegos es posible tener tal distribucion, es decir muchos juegos de precio bajo y varios de precio alto por esto no se hara un tratado especial

In [ ]:
sns.histplot(data=steam_games,x='Year',hue='Free to Play',bins=10,palette='rainbow',multiple='dodge')

plt.ylim(0,20000)
plt.show()

### 2015 es el año con mas juegos y mas juegos Free To Play

## Users Reviews

In [ ]:
user_review = pd.read_csv('./datasets/user_reviews.csv')

In [ ]:
user_review.sample(5)

In [ ]:
sns.histplot(data=user_review,x='sentiment_analysis',bins=10,palette='rainbow',hue='recommend')

plt.show()

### Como se ve del total de sentimiento malos, un alto porcentaje son recomendaciones falsas, y de los buenos el total de recomendaciones true son altas, Para ambas hay un porcentaje existente de recomendaciones que parecen opuesta al sentimiento

## Users Items

In [ ]:
with gzip.open('datasets/user_items_proc.csv.gz', 'rb') as f:
    user_items_proc = pd.read_csv(f, encoding='utf-8')

In [ ]:
user_items_proc.sample(5)

In [ ]:
user_items_proc['item_id'].nunique()

In [ ]:
user_items_proc.describe()

In [ ]:
sns.scatterplot(data=user_items_proc,x='items_count',y='playtime_forever')
plt.show()

### Se podria analisar de forma que a menos cantidad de items mas tiempo consume un usuario un juego especifico.

# Modelo de recomendacion

### Cargo el dataset usado para el modelo

In [2]:
user_reviews = pd.read_csv('./datasets/user_reviews.csv',usecols=['user_id','item_id','sentiment_analysis'])
user_reviews

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250,2
1,76561197970982479,22200,2
2,76561197970982479,43110,2
3,js41637,251610,2
4,js41637,227300,2
...,...,...,...
59300,76561198312638244,70,2
59301,76561198312638244,362890,2
59302,LydiaMorley,273110,2
59303,LydiaMorley,730,2


### Genero id numericos para user_id usando label encoder

In [3]:
label_encoder = LabelEncoder()
user_reviews['user_id_num'] = label_encoder.fit_transform(user_reviews['user_id'])
#elimino la columna user_id
user_reviews.drop(columns=['user_id'])
user_reviews = user_reviews[['user_id_num','item_id','sentiment_analysis']]

In [4]:
reader = Reader()
data = Dataset.load_from_df(user_reviews, reader)


### Separo el dataset en entrenamiento y testeo

In [5]:
train,test = train_test_splitSV(data,test_size = .25)

## Voy a usar un modelo De descomposicion en valor singular (SVD), el cual es un filtro colaborativo
### Voy a usar los datos de 
* user_id
* item_id
* sentiment_analysis: como rating

In [6]:
model = SVD()

### Aplico GridSearch para encontrar al modelo con los mejores hiperparametros

In [7]:
param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)
gs.fit(data)


In [8]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
best_model = gs.best_estimator['rmse']

0.7442449222206319
{'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


### Veo que el modelo tiene un rmse de 0.74 lo cual esta cerca de 1
### Ademas veo los mejores hiperparametros

## Por ultimo defino el modelo con los mejores hiperparametros y lo exporto

In [9]:
model = SVD(n_factors=5,n_epochs=20,lr_all=0.005,reg_all=0.2)
model.fit(train)

In [10]:
from surprise import accuracy
train_predictions = model.test(train.build_testset())
test_predictions = model.test(test)
print(f"RMSE en el conjunto de entrenamiento: {accuracy.rmse(train_predictions)}")
print(f"RMSE en el conjunto de prueba: {accuracy.rmse(test_predictions)}")

RMSE: 0.6664
RMSE en el conjunto de entrenamiento: 0.6663930199221555
RMSE: 0.7415
RMSE en el conjunto de prueba: 0.7414788453286719


### EL modelo obtuvo un mejor rendimiento en los datos de testeo

In [11]:
with open('./model/SVD_model.pkl', 'wb') as file: # Exporto mi modelo
    pickle.dump(model, file)